In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from selenium.common.exceptions import ElementNotVisibleException

In [30]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
list_news = soup.find('ul', class_='item_list')
last_news = list_news.find_all('li')[0]
link_news = last_news.find('a')['href']
print(link_news)

/news/8504/nasa-jpl-names-rolling-stones-rock-on-mars/


In [53]:
url = 'https://mars.nasa.gov' + link_news
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
title = soup.find('h1', class_='article_title')
title = title.text
title = title[1:]
title = title[:-1]
title

"NASA-JPL Names 'Rolling Stones Rock' on Mars"

In [46]:
par_list = soup.find('div', class_='wysiwyg_content')
# Get the first paragraph
par = par_list.find_all('p')[0]
para = par.text
para

"For decades, the music of The Rolling Stones has had a global reach here on Earth. Now, the band's influence extends all the way to Mars. The team behind NASA's InSight lander has named a Martian rock after the band: 'Rolling Stones Rock.'"

In [54]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [66]:
carousel_item = soup.find('article', class_="carousel_item")
footer = carousel_item.find('footer')
fig_id = footer.find('a')['data-link']
fig_id = fig_id.split("=")[1]
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/' + fig_id + '_ip.jpg'
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14254_ip.jpg'

In [67]:
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [70]:
stream = soup.find('ol', class_='stream-items js-navigable-stream')
stream2 = stream.find_all('li')[0]
stream3 = stream2.find('div', class_='js-tweet-text-container')
mars_weather = stream3.find('p').text
print (mars_weather)

InSight sol 261 (2019-08-21) low -102.4ºC (-152.4ºF) high -26.6ºC (-15.8ºF)
winds from the SSE at 4.9 m/s (11.0 mph) gusting to 16.0 m/s (35.8 mph)
pressure at 7.70 hPapic.twitter.com/MhPPOHJg3m


In [3]:
url = "https://space-facts.com/mars/"
tables = pd.read_html(url)
df = tables[1]
df.rename(columns={0:'Info',1:'Value'}, inplace=True)
df.set_index('info', inplace=True)
df

KeyError: 'info'

In [90]:
html_table = df.to_html()
html_table = html_table.replace('\n', "")
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>value</th>    </tr>    <tr>      <th>info</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [111]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
urls =[]
list = soup.find('div', class_='collapsible results')
lists = list.find_all('div', class_="item")

for item in lists:
    desc = item.find('div', class_='description')
    urls.append('https://astrogeology.usgs.gov' +  desc.find('a')['href'])
urls

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [113]:
hemi = []
for url in urls:
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    html = soup.find('div', class_='downloads')
    img = 'https://astrogeology.usgs.gov' + html.find('img')['src']
    title = soup.title.text
    title = title.split("|")[0]
    title = title.replace("Enhanced","")
    hemi.append({"title":title, "img": img})  
print(hemi)

[{'title': 'Cerberus Hemisphere  ', 'img': 'https://astrogeology.usgs.gov/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png'}, {'title': 'Schiaparelli Hemisphere  ', 'img': 'https://astrogeology.usgs.gov/cache/images/7677c0a006b83871b5a2f66985ab5857_schiaparelli_enhanced.tif_thumb.png'}, {'title': 'Syrtis Major Hemisphere  ', 'img': 'https://astrogeology.usgs.gov/cache/images/aae41197e40d6d4f3ea557f8cfe51d15_syrtis_major_enhanced.tif_thumb.png'}, {'title': 'Valles Marineris Hemisphere  ', 'img': 'https://astrogeology.usgs.gov/cache/images/04085d99ec3713883a9a57f42be9c725_valles_marineris_enhanced.tif_thumb.png'}]
